In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time

# データを保存するリスト
all_data = []
job_title = []
unit_price = []
required_experience = []
details_page_link = []
representative_name = []
headquarters_locations = []
industry = []
contact_number = []
company_info_link = []
all_data = []

# URLリスト
urls = [
    'https://next.rikunabi.com/rnc/docs/cp_s00700.jsp?page={page_number}&cur=ACgAAQAoAAAAAAAAAAAAAAACMT4N4gECAQkUBgU6vVJcMq7CpmpPjbzo3HJe4dF7UXdwgM%2BXoqqxn4oQb50yRo6ag3iAKyyeqyFDcEzcwkGZDsn8prbqAeX62tWArTu1z3hoh%2FE%3D&cur_p=2&occupation_cd=EHPW9&wrk_plc_long_cd=0313113103&wrk_plc_long_cd=0313113105&wrk_plc_long_cd=0313113106&wrk_plc_long_cd=0313113107&wrk_plc_long_cd=0313113108&wrk_plc_long_cd=0313113109&wrk_plc_long_cd=0313113110&wrk_plc_long_cd=0313113112&wrk_plc_long_cd=0313113113&wrk_plc_long_cd=0313113114&wrk_plc_long_cd=0313113115&wrk_plc_long_cd=0313113116&wrk_plc_long_cd=0313113117&wrk_plc_long_cd=0313113118&wrk_plc_long_cd=0313113119&wrk_plc_long_cd=0313113120&wrk_plc_long_cd=0313113121&wrk_plc_long_cd=0313113122&wrk_plc_long_cd=0313113123&employ_frm_cd=01',
    'https://next.rikunabi.com/rnc/docs/cp_s00700.jsp?page={page_number}&cur=ACgAAQAoAAAAAAAAAAAAAAACMS3oJwECAQcQBwDJ0q1coF8J1A%2BXbx8LfMoUFa0H0ozwxuh1Cl2Yt6nVnVjXIG6JTdmzSszrVc3%2FqoFVpTVY9trd%2FqqkEQbUcIETWz%2BHIPfek7ZmfQ24OGBs&cur_p=2&occupation_cd=EHPW9&wrk_plc_long_cd=0308000000&wrk_plc_long_cd=0309000000&wrk_plc_long_cd=0310000000&wrk_plc_long_cd=0311000000&wrk_plc_long_cd=0312000000&wrk_plc_long_cd=0312100000&wrk_plc_long_cd=0313000000&wrk_plc_long_cd=0313113101&wrk_plc_long_cd=0313113102&wrk_plc_long_cd=0313113104&wrk_plc_long_cd=0313113111&wrk_plc_long_cd=0319000000&employ_frm_cd=01',
    'https://next.rikunabi.com/rnc/docs/cp_s00700.jsp?page={page_number}&cur=ACgAAQAoAAAAAAAAAAAAAAACMS94hwEBAREOFTkoHA8qete%2FaHxbbczb6YgIKotlcDsgj7i93qrrM1Y2G%2Fqv49iBIYNIQQA5t16cEfjNkAQfjTL%2F5QZCjB65EDc2vY%2BpdVFqGleeG3XeZivY3Q%3D%3D&cur_p=2&occupation_cd=EHPW9&wrk_plc_long_cd=0100000000&wrk_plc_long_cd=0200000000&wrk_plc_long_cd=0314000000&wrk_plc_long_cd=0314100000&wrk_plc_long_cd=0314130000&wrk_plc_long_cd=0400000000&wrk_plc_long_cd=0500000000&wrk_plc_long_cd=0625025201&wrk_plc_long_cd=0625025202&wrk_plc_long_cd=0625025203&wrk_plc_long_cd=0625025204&wrk_plc_long_cd=0625025206&wrk_plc_long_cd=0625025207&wrk_plc_long_cd=0625025208&wrk_plc_long_cd=0625025210&wrk_plc_long_cd=0625025211&wrk_plc_long_cd=0625025212&wrk_plc_long_cd=0625025213&wrk_plc_long_cd=0625025214&wrk_plc_long_cd=0625025380&wrk_plc_long_cd=0625025420&wrk_plc_long_cd=0625025440&wrk_plc_long_cd=0626000000&wrk_plc_long_cd=0626100000&employ_frm_cd=01',
    'https://next.rikunabi.com/rnc/docs/cp_s00700.jsp?page={page_number}&cur=ACgAAQAoAAAAAAAAAAAAAAACMTaFXwECAQgSBhev3K1jV%2Fhq0UZwY%2BX1UqihpAGtcNhqLyTFm%2B9%2F7sDZRROGhG50JsJlEACLpDnR7sM3Z6xhMuW1uaF7adHbyHzYuWQV8uYV2cu4g2xbxMhTmG2Cdg%3D%3D&cur_p=2&occupation_cd=EHPW9&wrk_plc_long_cd=0627000000&wrk_plc_long_cd=0627100000&wrk_plc_long_cd=0628000000&wrk_plc_long_cd=0628100000&wrk_plc_long_cd=0629000000&wrk_plc_long_cd=0630000000&wrk_plc_long_cd=0700000000&wrk_plc_long_cd=0800000000&employ_frm_cd=01'
]

# ユーザーエージェントのリスト
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:55.0) Gecko/20100101 Firefox/55.0', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko'
]

def fetch_page_data(url, page_number):
    try:
        headers = {'User-Agent': random.choice(user_agents)}
        response = requests.get(url.format(page_number=page_number), headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # 各求人情報を取得
        job_elements = soup.select("li.rnn-jobOfferList__item")
        total_jobs = len(job_elements)

        page_data = []

        # 各求人情報をループ
        for job_element in job_elements:
            # 各情報を取得
            title = job_element.select_one("h2.rnn-textLl.js-abScreen__title").text.strip()

            try:
                income_details = job_element.select_one("tr.js-abScreen__income td.rnn-offerDetail__text").text.strip()
                income_details_cleaned = ' '.join(income_details.split())
            except:
                income_details_cleaned = ''

            try:
                requirements = job_element.select_one("tr.js-abScreen__prefer td.rnn-offerDetail__text").text.strip()
            except:
                requirements = ''

            # 詳細ページのリンクを取得
            link = job_element.select_one("a.rnn-linkText.rnn-linkText--black")['href']
            details_url = f'https://next.rikunabi.com{link}'
            details_response = requests.get(details_url, headers=headers)
            details_soup = BeautifulSoup(details_response.content, 'html.parser')

            # 「リクルートエージェントからの求人」が含まれているか確認
            if 'リクルートエージェントからの求人' in details_soup.get_text():
                company_name = details_soup.select_one('p.rnn-offerCompanyName').text.strip()

                table_rows = details_soup.select('table.rnn-detailTable tr.rnn-tableGrid')

                def get_value_from_table(row_title):
                    for row in table_rows:
                        th_element = row.select_one('th')
                        if th_element and th_element.text.strip() == row_title:
                            td_element = row.select_one('td')
                            return td_element.text.strip() if td_element else ''
                    return ''

                ceo_name = get_value_from_table('代表者')
                headquarters_location = get_value_from_table('本社所在地')
                industry_heading = get_value_from_table('業種')
                contact_info_text = get_value_from_table('連絡先')
                url = details_url
            else:
                company_info = details_soup.select_one('div.rn3-companyOfferCompany')

                try:
                    company_name = company_info.find('h3', string='社名').find_next_sibling('p', class_='rn3-companyOfferCompany__text').get_text(strip=True).replace('\xa0', ' ').replace('\n', ' ')
                except:
                    company_name = ''

                try:
                    ceo_name = company_info.find('h3', string='代表者').find_next_sibling('p', class_='rn3-companyOfferCompany__text').get_text(strip=True).replace('\xa0', ' ').replace('\n', ' ')
                except:
                    ceo_name = ''

                try:
                    headquarters_location = company_info.find('h3', string='本社所在地').find_next_sibling('p', class_='rn3-companyOfferCompany__text').get_text(strip=True).replace('\xa0', ' ').replace('\n', ' ')
                except:
                    headquarters_location = ''

                try:
                    industry_headings = company_info.find('h3', string='業種').find_next_sibling('p', class_='rn3-companyOfferCompany__text').get_text(strip=True).replace('\xa0', ' ').replace('\n', ' ')
                    parts = industry_headings.split('/')
                    industry_heading = parts[0].replace('\xa0', ' ').replace('\n', ' ')
                except:
                    industry_heading = ''

                try:
                    contact_heading = details_soup.find('h3', class_='rn3-companyOfferEntry__heading', string='連絡先')
                    contact_info_div = contact_heading.find_next_sibling('div')
                    contact_info_text = contact_info_div.get_text(separator='\n', strip=True)
                    contact_info_text = contact_info_text.replace('\xa0', ' ')
                    contact_info_text = ' '.join(contact_info_text.split())
                    page = contact_info_div.find('a')
                    url = 'https://next.rikunabi.com/' + page['href']
                except:
                    url = ''
                    contact_info_text = ''

            # リストに情報を追加
            data = {
                "job_title": title,
                "company_name": company_name,
                "contact_number": contact_info_text,
                "industry": industry_heading,
                "representative_name": ceo_name,
                "company_info_link": url,
                "unit_price": income_details_cleaned,
                "headquarters_location": headquarters_location,
                "qualification": '',  # qualification の情報がどこにあるかによって変更
                "required_experience": requirements,
                "details_page_link": details_url,
                "contact_url": url
            }

            page_data.append(data)
        
        # ランダムな遅延を追加
        time.sleep(random.uniform(3, 6))  # 遅延時間を増やす
        
        return page_data
    except Exception as e:
        print(f"Error processing page {page_number}: {e}")
        return []

# ページ番号を1から96までループ
page_numbers = range(1, 93)
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_page = {executor.submit(fetch_page_data, url, page_number): (url, page_number) for url in urls for page_number in page_numbers}
    for future in tqdm(as_completed(future_to_page), total=len(future_to_page), desc="Processing pages"):
        page_data = future.result()
        all_data.extend(page_data)

# データフレームに変換
df1 = pd.DataFrame(all_data)
df1.to_csv('rikunavi.csv', index=False, encoding='utf-8')

Processing pages: 100%|██████████| 368/368 [53:42<00:00,  8.76s/it]  
